In [12]:
import scrapy
import os
import re
import requests

from scrapy.selector import Selector
from scrapy.http import Request


class YoutubeSpider(scrapy.Spider):
    name = '@PW-Foundation'
    allowed_domains = ['www.youtube.com']

    def __init__(self, query=None, *args, **kwargs):
        super(YoutubeSpider, self).__init__(*args, **kwargs)
        self.start_urls = [f'https://www.youtube.com/{query}/videos']

    def parse(self, response):
        sel = Selector(response)
        allchannellist = sel.css('a#video-title-link')
        urls = list(set(allchannellist.css('::attr(href)').extract()))

        urls = urls[:5]
        thumbnails_links = []

        for url in urls:
            exp = "^.*((youtu.be\/)|(v\/)|(\/u\/\w\/)|(embed\/)|(watch\?))\??v?=?([^#&?]*).*"
            s = re.findall(exp, url)[0][-1]
            thumbnail = f"https://i.ytimg.com/vi/{s}/maxresdefault.jpg"
            thumbnails_links.append(thumbnail)

            # download images
            os.makedirs('image', exist_ok=True)
            os.chdir('image')
            with open(f'{s}.jpg', 'wb') as f:
                im = requests.get(thumbnail)
                f.write(im.content)

        all_titles = sel.css('a#video-title-link')
        titles = all_titles.css('::attr(title)').extract()[:5]

        all_views_time = sel.css('span.style-scope.ytd-video-meta-block::text').extract()
        views_time = all_views_time[0:14]
        views = [i for i in views_time if i.endswith("views")]
        times = [i for i in views_time if not i.endswith("views")]

        mydict = {"urls": urls, "thumbnails_links": thumbnails_links, 
                  "titles": titles, "views": views, "time": times}
        yield mydict


In [17]:
!scrapy crawl @PW-Foundation -o output.json

Scrapy 2.8.0 - no active project

Unknown command: crawl

Use "scrapy" to see available commands


In [13]:
mydict

NameError: name 'mydict' is not defined